# Conexión por SSH

Inicializar la conexion ssh


In [16]:
import paramiko
import getpass

host = "158.251.88.197"
port = 22
username = "gonzalo"
password = getpass.getpass()

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

`loadBoxes(path,instance)`
--
Se carga las cajas de una instancia

In [5]:
import numpy as np

dimBox = dict()
vol = dict()
volBin = dict() #Volumen de cada bin

def loadBoxes(idClass, boxSize, idInstance):
    filePath = '../benchs/class'+str(idClass)+'/'+str(boxSize)+'.txt'
    instanceIndex = -1
    countBox = 0
    totalBox = 0
    boxesinfo = []
    binDim = []
    
    boxesFile = open(filePath,'r')
    
    #Lectura del archivo
    for line in boxesFile.readlines():
        info = line.strip().split()
        
        if(instanceIndex < idInstance and len(info) == 4):
            instanceIndex+=1
            totalBox = int(info[0])
        
        elif(instanceIndex == idInstance):
            #La instancia que se esta buscando
            if(len(info) == 4):
                instanceIndex+=1
                totalBox = int(info[0])
                binDim = [int(info[i]) for i in range(1, len(info))]
                binDim = np.array(binDim)
            
            elif(len(info) == 3 and countBox < totalBox):
                countBox+=1    
                b = np.array([int(info[i]) for i in range(0, len(info))])
                b.sort()
                boxesinfo.append(b)

    countBox = 0
    for boxSize in boxesinfo:
        dimBox[countBox] = np.array(boxSize)
        countBox +=1


Implementé un *servidor MCLP*, el cual escucha instrucciones para generar bins.

El siguiente código lanza el servidor en el background, carga una instancia y lo deja escuchando peticiones por el puerto 8080

In [6]:
def loadInstance(idClass=1, boxSize=100, idInstance=0, port=8061):
    transport = ssh.get_transport()
    channel = transport.open_session()
    loadBoxes(idClass, boxSize, idInstance)
    channel.exec_command("killall BASIC_BSG_MCLP; ./mclp/Metasolver/BASIC_BSG_MCLP mclp/Metasolver/problems/mclp/benchs/class"+ \
                         str(idClass)+"/"+str(boxSize)+".txt -i "+str(idInstance)+" --port="+str(port))
    return dimBox, channel

Para cerrar el servidor MCLP

# Multiple Container Loading Problem

In [7]:
volBin = dict() #Volumen de cada bin

def generate_bins(port=8060):
    #probarlo con distintas instancias
    
    local_command = "echo generate_bins | netcat localhost "+str(port)
    stdin, stdout, stderr = ssh.exec_command(local_command)
    flag = False
    bins = []
    count = 0
    for line in stdout.readlines():
        if "BINS:" in line: 
            flag = True
            continue
        if flag:    
            #Modificar igual que generate_bin
            line = line.strip().split()
            vol = float(line[0]) #Porcentaje de llenado del bin
            newBin = np.array([int(line[x]) for x in range(1,len(line))])
            #calcular el volumen
            bins.append(newBin)
            volBin[count] = vol
            count +=1
    #recordar forma del arreglo        
    return bins

**Construcción de bin priorizando cajas (BSG)**

In [17]:
import paramiko
import getpass
import numpy as np

def generate_bin(boxes, new_box, port=8060):
    global channel
    index = 0
    leftboxes = []
    newBin = []
    boxes = np.insert(boxes, 1, new_box)
    boxes_str = [str(box) for box in boxes]
    pboxes_str = str(new_box)

    local_command = "echo generate_bin " + " ".join(boxes_str) + " -1 " + pboxes_str +  " -2 | netcat localhost "+str(port)
    stdin, stdout, stderr = ssh.exec_command(local_command)
    lines = stdout.readlines()
    print(lines)
    
    for line in lines: 
        #ajustar linea para almacenar los conjuntos de cajas
        line = line.strip().split()
        
        if(index == 0):
            #primera linea boxes 
            percent = float(line.pop(0))
            newBin = np.array([int(x) for x in line])            
        else:
            #segunda linea left boxes
            leftboxes = np.array([int(x) for x in line])
        index+=1

    return newBin, leftboxes

In [18]:
boxes, channel = loadInstance(1,100,0,port=8061)   
bins = generate_bins(port=8061)
print("Universe")
print (bins)
print("Generator")
generate_bin([ 0, 63, 83, 89], 62, port=8061)
stdin, stdout, stderr = ssh.exec_command("echo get_boxes | netcat localhost 8061")
sol_vector = []
for line in stdout.readlines():
    box = np.fromstring(line, sep=',', dtype=int)
    if len(box)>1:
        sol_vector.append(box)

Universe
[array([ 0, 63, 83, 89]), array([ 1,  8, 26, 69, 73, 84, 90]), array([ 2, 30, 50, 92, 93]), array([3, 5]), array([ 4, 32, 54, 88]), array([ 6, 21, 64, 65, 66, 94, 97, 98]), array([ 7, 46, 96]), array([ 9, 10, 11, 12, 15, 20, 23, 34, 35, 39, 48, 49, 55, 62, 77, 99]), array([13, 31, 51, 52, 68, 80]), array([14, 29, 44, 81]), array([16, 38, 56, 60]), array([17, 22, 25, 33, 79]), array([18, 37]), array([19, 61, 70, 95]), array([24, 57, 86]), array([27, 36, 41, 72]), array([28, 91]), array([40, 53, 74]), array([42, 43, 82]), array([45, 75, 85]), array([47, 67]), array([58, 71, 76]), array([59, 87]), array([78])]
Generator
['0.7891 0 62 63 83 89 \n', '\n']


In [2]:
import box_plotter
from box_plotter import Box, save_container
import importlib
importlib.reload(box_plotter)

save_container([100,100,100], sol_vector, 'fig1.png')

from IPython.display import Image 
pil_img = Image(filename='fig1.png')
display(pil_img)

NameError: name 'box_plotter' is not defined